# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
# Write your code below.

# Load environment variables from a .env file
%load_ext dotenv
%dotenv



In [2]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [2]:
import os
from glob import glob

# Write your code below.
import dask.dataframe as dd

# Load the PRICE_DATA environment variable
PRICE_DATA = os.getenv("PRICE_DATA")

# Use glob to find all parquet files under PRICE_DATA (recursively)
parquet_files = glob(os.path.join(PRICE_DATA, "**", "*.parquet"), recursive=True)

# Confirm results
print(f"PRICE_DATA: {PRICE_DATA}")
print(f"Number of parquet files found: {len(parquet_files)}")



PRICE_DATA: ../../05_src/data/prices/
Number of parquet files found: 3083


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [ ]:
import numpy as np
import dask.dataframe as dd

# Assumes `parquet_files` is already defined.
dd_px = dd.read_parquet(parquet_files)

# Normalize adjusted-close column name if needed
if "Adj_Close" not in dd_px.columns and "Adj Close" in dd_px.columns:
    dd_px = dd_px.rename(columns={"Adj Close": "Adj_Close"})

# Ensure Date is datetime64[ns] (no-op if already correct)
if dd_px["Date"].dtype.kind not in ("M",):
    dd_px = dd_px.assign(Date=dd_px["Date"].astype("M8[ns]"))

# ---- Build explicit meta (schema) for the output with the four new columns ----
meta = (
    dd_px._meta.assign(
        Close_lag_1=np.float64(),
        Adj_Close_lag_1=np.float64(),
        returns=np.float64(),
        hi_lo_range=np.float64(),
    )
)

# ---- Per-ticker feature logic: sort by Date, add lags/returns/range ----
def _per_ticker_features(pdf):
    # NOTE: This function is executed by Dask on each group partition;
    # the returned result is stitched back into a single Dask DataFrame.
    pdf = pdf.sort_values("Date")
    pdf["Close_lag_1"] = pdf["Close"].shift(1)
    pdf["Adj_Close_lag_1"] = pdf["Adj_Close"].shift(1)
    pdf["returns"] = pdf["Close"] / pdf["Close_lag_1"] - 1
    pdf["hi_lo_range"] = pdf["High"] - pdf["Low"]
    return pdf

# Group by ticker and apply the feature function with explicit meta
dd_feat = dd_px.groupby("ticker", group_keys=False).apply(_per_ticker_features, meta=meta)

# Quick peek 
dd_feat.head()


,Date,Open,High,Low,Close,Adj_Close,Volume,source,ticker,Year,Close_lag_1,Adj_Close_lag_1,returns,hi_lo_range


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [ ]:
# Write your code below.


# Convert the Dask DataFrame to a pandas DataFrame
# Assumes `dd_feat` exists from the previous step
pdf_feat = dd_feat.reset_index().compute()

# Ensure chronological order within each ticker
pdf_feat = pdf_feat.sort_values(["ticker", "Date"])

# Add a 10-day moving average of returns (per ticker)
#  transform keeps the result aligned to the original rows
pdf_feat["returns_ma10"] = (
    pdf_feat.groupby("ticker")["returns"]
            .transform(lambda s: s.rolling(window=10, min_periods=10).mean())
)

# Optional quick check
pdf_feat.head()




,index,Date,Open,High,Low,Close,Adj_Close,Volume,source,ticker,Year,Close_lag_1,Adj_Close_lag_1,returns,hi_lo_range,returns_ma10
0,137315,2001-07-19,15.10,15.29,15.00,15.17,11.404394,34994300.0,ACN.csv,ACN,2001,NaN,NaN,NaN,0.29,NaN
1,137316,2001-07-20,15.05,15.05,14.80,15.01,11.284108,9238500.0,ACN.csv,ACN,2001,15.17,11.404394,-0.010547,0.25,NaN
2,137317,2001-07-23,15.00,15.01,14.55,15.00,11.276587,7501000.0,ACN.csv,ACN,2001,15.01,11.284108,-0.000666,0.46,NaN
3,137318,2001-07-24,14.95,14.97,14.70,14.86,11.171341,3537300.0,ACN.csv,ACN,2001,15.00,11.276587,-0.009333,0.27,NaN
4,137319,2001-07-25,14.70,14.95,14.65,14.95,11.238999,4208100.0,ACN.csv,ACN,2001,14.86,11.171341,0.006057,0.30,NaN


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
No. Dask can compute a moving average. 
Converting to pandas was a pragmatic choice for simplicity and quick validation, but it isn’t strictly required for this feature.
+ Would it have been better to do it in Dask? Why?
Yes, in the case where data size is large or you want to stay scalable. 
1. Scalability & memory: 
Dask: Keeps computation lazy and chunked by partitions. 
pandas: Fine for small/medium data, but it breaks scalability and risks memory pressure.

2. Performance & parallelism
Dask: Rolling operations can be parallelized across partitions. W
pandas: Runs eagerly on one process; you lose the benefits of the Dask task graph you built up to this point.

3. Operational consistency
Keeping the whole pipeline in Dask means one execution model (lazy graph → write to Parquet), consistent dtypes/schemas, and easier productionization (no sudden memory spikes from .compute()).





(1 pt)

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.